# Fast GP model evaluation

For visualization in tools like Badger, same model is frequently reused many times. It is possible to use advanced
PyTorch functionality to speed up the evaluation of the model and acquisition function. This comes with many caveats
that limit the flexibility and dynamic/conditional features. As a general rule, if any non-tensor model parameter is
changed, objects must be recompiled.

In [ ]:
from copy import deepcopy
from xopt import Xopt, Evaluator
from xopt.generators.bayesian import UpperConfidenceBoundGenerator
from xopt.resources.test_functions.tnk import evaluate_TNK, tnk_vocs

# Ignore all warnings
import warnings

warnings.filterwarnings("ignore")

vocs = deepcopy(tnk_vocs)
vocs.objectives = {"y2": "MINIMIZE"}

generator = UpperConfidenceBoundGenerator(vocs=vocs)
generator.use_cuda = True

evaluator = Evaluator(function=evaluate_TNK)

X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
generator = X.generator

In [ ]:
# add a lot of points, slowing down model
X.random_evaluate(1000);

In [ ]:
X.generator.train_model();

In [ ]:
m = deepcopy(X.generator.model)

In [ ]:
fig, ax = X.generator.visualize_model(
    n_grid=30, output_names=["y2"], show_acquisition=False, show_samples=False
)

In [ ]:
X.generator.model = m

In [ ]:
# first call does tracing
fig, ax = X.generator.visualize_model(
    n_grid=30,
    model_compile_mode="trace",
    output_names=["y2"],
    show_acquisition=False,
    show_samples=False,
)

In [ ]:
# second invocation uses a pre-traced model and is a tiny bit faster (most of walltime is used by plotting)
fig, ax = X.generator.visualize_model(
    n_grid=30,
    model_compile_mode="trace",
    output_names=["y2"],
    show_acquisition=False,
    show_samples=False,
)